## Feature Engineering & Model Training


In [98]:
import numpy as np
import pandas as pd


In [99]:
# Data Ingestion
df = pd.read_csv('data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [100]:
# Drop id column
df = df.drop(labels=['id'], axis=1)
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   carat    193573 non-null  float64
 1   cut      193573 non-null  object 
 2   color    193573 non-null  object 
 3   clarity  193573 non-null  object 
 4   depth    193573 non-null  float64
 5   table    193573 non-null  float64
 6   x        193573 non-null  float64
 7   y        193573 non-null  float64
 8   z        193573 non-null  float64
 9   price    193573 non-null  int64  
dtypes: float64(6), int64(1), object(3)
memory usage: 14.8+ MB


In [102]:
# Split into independent and dependent features
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [103]:
# Segregating Numerical and Categorical features
numerical_features = [feature for feature in X.columns if X[feature].dtypes !=object]
categorical_features = [feature for feature in X.columns if X[feature].dtypes ==object]

In [104]:
numerical_features

['carat', 'depth', 'table', 'x', 'y', 'z']

In [105]:
categorical_features

['cut', 'color', 'clarity']

In [106]:
# Define custom ranking for each ordinal values
cut_categories = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']


## **Make Pipeline**


In [107]:
from sklearn.impute import SimpleImputer # For Handling Missing Values
from sklearn.preprocessing import StandardScaler # For Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # For Ordinal Encoding

## Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [108]:
# Numerical Pipeline
num_pipepline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)


In [109]:
# Categorical Pipeline
cat_pipepline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder', OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories])),
        ('scaler', StandardScaler())
    ]
)


In [110]:
preprocessor = ColumnTransformer([
    ('num_pipepline',num_pipepline,numerical_features),
    ('cat_pipepline', cat_pipepline, categorical_features)
])


In [111]:
# Train, Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,random_state=42,shuffle=True)


In [112]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=preprocessor.get_feature_names_out())

In [113]:
X_train.head(2)


,num_pipepline__carat,num_pipepline__depth,num_pipepline__table,num_pipepline__x,num_pipepline__y,num_pipepline__z,cat_pipepline__cut,cat_pipepline__color,cat_pipepline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.874100,-0.936747,1.350746
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-1.137644,0.910853,0.684455


In [114]:
X_test.head(2)


,num_pipepline__carat,num_pipepline__depth,num_pipepline__table,num_pipepline__x,num_pipepline__y,num_pipepline__z,cat_pipepline__cut,cat_pipepline__color,cat_pipepline__clarity
0,-0.629077,0.258230,-0.12063,-0.600482,-0.581521,-0.572248,0.874100,-1.552614,-0.648127
1,2.605374,-2.148014,-0.12063,2.126042,2.198832,1.959219,-1.137644,0.294987,-1.314417


In [115]:
# Model Training
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [116]:
regression = LinearRegression()
regression.fit(X_train, y_train)


LinearRegression()

In [117]:
y_pred = regression.predict(X_test)


In [118]:
r2_score(y_test, y_pred)


0.9362906819996045

In [119]:
# Root Mean Square Error
np.sqrt(mean_squared_error(y_test, y_pred))

1014.6296630375483

In [120]:

# Function to evaluate the model
def evaluate_model(test, pred):
    mae = mean_absolute_error(test, pred)
    mse = mean_squared_error(test, pred)
    rmse = np.sqrt(mse)
    r2_square = r2_score(test, pred)

    return mae, rmse, r2_square


In [121]:
# Train Multiple Model
models = {
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}


In [122]:
for x in models.values():
    print(x)

LinearRegression()
Lasso()
Ridge()
ElasticNet()


In [124]:
for x in models.values():
    model = x
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    mae, rmse, r2_square = evaluate_model(y_test, pred)

    print(f'Model Name : {x}')
    print("Model Training Performance")
    print("RMSE : ", rmse)
    print("MAE : ", mae)
    print("R2 Score : ", r2_square*100)

    print('='*50)
    print('\n')


Model Name : LinearRegression()
Model Training Performance
RMSE :  1014.6296630375483
MAE :  675.0758270067445
R2 Score :  93.62906819996046


Model Name : Lasso()
Model Training Performance
RMSE :  1014.6591302750638
MAE :  676.2421173665508
R2 Score :  93.62869814082755


Model Name : Ridge()
Model Training Performance
RMSE :  1014.6343233534424
MAE :  675.1077629781317
R2 Score :  93.62900967491629


Model Name : ElasticNet()
Model Training Performance
RMSE :  1533.3541245902313
MAE :  1060.9432977143008
R2 Score :  85.44967219374031


